In [1]:
import findspark
import os
import gc
#findspark.init() 
SPARK_HOME='/opt/cloudera/parcels/CDH/lib/spark'
findspark.init(SPARK_HOME)

import time
import math
import copy
import csv
import json
import os
import codecs
import subprocess
import numpy as np
from pyspark import SQLContext
from pyspark.sql import Row
from pyspark.sql import functions as F
from pyspark.sql.functions import size, array_union,flatten,array_sort,coalesce,broadcast,collect_list, collect_set, udf, array_remove, log, lit, first, col, array, sort_array,split, explode, desc, asc, row_number,isnan, when, count
from pyspark.sql.types import *
import rtree
from pyspark.sql import Window
import geofeather
from pyspark.storagelevel import StorageLevel

from pyspark.sql import SparkSession
from pyspark import StorageLevel
import geopandas as gpd
import pandas as pd
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, LongType, FloatType, ArrayType, MapType
from shapely.geometry import Point
from shapely.geometry import Polygon

from sedona.register import SedonaRegistrator
from sedona.core.SpatialRDD import SpatialRDD, PointRDD, CircleRDD, PolygonRDD, LineStringRDD
from sedona.core.enums import FileDataSplitter
from sedona.utils.adapter import Adapter
from sedona.core.spatialOperator import KNNQuery
from sedona.core.spatialOperator import JoinQuery
from sedona.core.spatialOperator import JoinQueryRaw
from sedona.core.spatialOperator import RangeQuery
from sedona.core.spatialOperator import RangeQueryRaw
from sedona.core.formatMapper.shapefileParser import ShapefileReader
from sedona.core.formatMapper import WkbReader
from sedona.core.formatMapper import WktReader
from sedona.core.formatMapper import GeoJsonReader
from sedona.sql.types import GeometryType
from sedona.core.enums import GridType
from sedona.core.SpatialRDD import RectangleRDD
from sedona.core.enums import IndexType
from sedona.core.geom.envelope import Envelope
from sedona.utils import SedonaKryoRegistrator, KryoSerializer

os.environ['PYSPARK_PYTHON'] = "./environment/bin/python"
#os.environ['PYSPARK_PYTHON'] = "/home/qiany/.conda/envs/py37/bin/python"
os.environ['YARN_CONF_DIR'] = "/opt/cloudera/parcels/CDH/lib/spark/conf/yarn-conf"

In [2]:
tin_file = input("Here is a programe to extract boundary relations, please input the absolute or relative path to your .ts file:")

# get the directory, basename of the input file
print("\n********************")
tin_directory = os.path.dirname(tin_file)
print("tin_directory: ", tin_directory)

directory_type = input("Is the data stored in hdfs(0) or Tri_data(1) or Tetra_data (2):") or "2"

if directory_type == '0':
    directory = 'hdfs_data'
elif directory_type == '1':
    directory = 'Tri_data'
else:
    directory = 'Tetra_data'
    
tin_basename = os.path.basename(tin_file) # input_vertices_2.off
print("tin_basename: ", tin_basename)

tin_filename = os.path.splitext(tin_basename)[0] # input_vertices_2
print("tin_filename: ", tin_filename)

tin_extension = os.path.splitext(tin_basename)[1] # .off
print("tin_extension: ", tin_extension)

print("\n********************")
print("This is a TIN file in \"%s\" format" % tin_extension)

filtra = 'yes'

# allocate the number of executors, the number of cores per executor, and the amount of memory per executor
Num_executor = '64'
Num_core_per_executor = '5'
Memory_executor = '64g'
MemoryOverhead_executor = '8g'

# allocate the number of cores for the driver node
Num_core_per_driver = '5'
Memory_driver = '64g'
MemoryOverhead_driver = '32g'

# the default number of shuffle partitions
Num_shuffle_partitions = input("spark.sql.shuffle.partitions:")


********************
tin_directory:  /home/qiany/yuehui/pyspark/Tetra_mesh/data
tin_basename:  Lander_small.ts
tin_filename:  Lander_small
tin_extension:  .ts

********************
This is a TIN file in ".ts" format


In [3]:
'''
spark.executor.cores: # Number of concurrent tasks an executor can run, euqals to the number of cores to use on each executor
spark.executor.instances: # Number of executors for the spark application
spark.executor.memory: # Amount of memory to use for each executor that runs the task
spark.executor.memoryOverhead:
spark.driver.cores: # Number of cores to use for the driver process; the default number is 1
spark.driver.memory: # Amount of memory to use for the driver
spark.driver.maxResultSize: to define the maximum limit of the total size of the serialized result that a driver can store for each Spark collect action
spark.default.parallelism: # Default number of partitions in RDDs returned by transformations like join, reduceByKey, and parallelize when not set by user. It can be set as spark.executor.instances * spark.executor.cores * 2
spark.sql.shuffle.partitions: determine how many partitions are used when data is shuffled between nodes, e.g., joins or aggregations. usually 1~5 times of executor.instances * executor.cores
spark.memory.storageFraction: determines the fraction of the heap space that is allocated to caching RDDs and DataFrames in memory.
spark.kryoserializer.buffer.max: determine the maximum of data that can be serialized at once; this must be larger than any object we attempt to serialize
spark.rpc.message.maxSize: # Maximum message size (in MiB) to allow in "control plane" communication; generally only applies to map output size information sent between executors and the driver. To communicate between the nodes, Spark uses a protocol called RPC (Remote Procedure Call), which sends messages back and forth. The spark.rpc.message.maxSize parameter limits how big these messages can be. 
spark.sql.broadcastTimeout: Spark will wait for this amount of time before giving up on broadcasting a table. Broadcasting can take a long time if the table is large or if there is a shuffle operation before it.
spark.sql.autoBroadcastJoinThreshold: Spark will broadcast a table to all worker nodes when performing a join if its size is less than this value; -1 means disabling broadcasting
'''

date = time.strftime("%m,%d,%Y")
date_name = date.split(',')[0] + date.split(',')[1] + date.split(',')[2]

hour = time.strftime("%H,%M")
hour_name = hour.split(',')[0] + hour.split(',')[1]

Topo = input("What kind of connectivity relation do you want to retrieve?")

spark_app_name = "TopoRela_Boundary_TetraMesh_" + tin_filename + '_' + date_name + '_' + hour_name + '_' + Topo
print("spark_app_name:", spark_app_name)

spark = SparkSession \
.builder \
.appName(spark_app_name) \
.master('yarn') \
.config("spark.serializer", KryoSerializer.getName) \
.config('spark.jars','sedona-core-2.4_2.11-1.0.0-incubating.jar,sedona-sql-2.4_2.11-1.0.0-incubating.jar,sedona-python-adapter-2.4_2.11-1.0.0-incubating.jar,sedona-viz-2.4_2.11-1.0.0-incubating.jar,geotools-wrapper-geotools-24.0.jar,graphframes-0.8.0-spark2.4-s_2.11.jar') \
.config('spark.executor.cores', Num_core_per_executor) \
.config('spark.executor.instances', Num_executor) \
.config('spark.executor.memory', Memory_executor) \
.config('spark.executor.memoryOverhead', MemoryOverhead_executor) \
.config('spark.driver.cores', Num_core_per_driver) \
.config('spark.driver.memory', Memory_driver) \
.config('spark.driver.memoryOverhead', MemoryOverhead_driver) \
.config('spark.driver.maxResultSize', '0') \
.config('spark.dynamicAllocation.enabled', 'false') \
.config('spark.network.timeout', '10000001s') \
.config('spark.executor.heartbeatInterval', '10000000s') \
.config('spark.sql.shuffle.partitions', Num_shuffle_partitions) \
.config("spark.default.parallelism", '400') \
.config("spark.kryoserializer.buffer.max", "1024mb") \
.config('spark.rpc.message.maxSize', '256') \
.config("spark.sql.broadcastTimeout", "36000") \
.config("spark.sql.autoBroadcastJoinThreshold", "-1") \
.config("spark.sql.objectHashAggregate.sortBased.fallbackThreshold", "-1") \
.config('spark.yarn.dist.archives', '/local/data/yuehui/py37.tar.gz#environment') \
.config("spark.python.profile", "true") \
.config("spark.eventLog.enabled", "true") \
.config("spark.eventLog.logStageExecutorMetrics", "true") \
.getOrCreate()

What kind of connectivity relation do you want to retrieve? TV


spark_app_name: TopoRela_Boundary_TetraMesh_Lander_small_06192024_0126_TV


### read input files

In [4]:
# read_tetra_order() is a function used to read tetrahedron from a csv file
def read_tetra_order(hdfs_tetra_origin):
    '''
    this function has two input parameters.
    filtra: 'yes' or 'no', yes means that the input csv file is ordered by default
    directory: a string denoting the directory to a tetrahedra file
    tin_filename: a string denoting the file name of a tetrahedra extension, e.g., 827_monviso
    '''
        
    schema_tetra_origin = StructType([ \
        StructField("tetra_order",IntegerType(),True), \
        StructField("r1",IntegerType(),True), \
        StructField("r2",IntegerType(),True), \
        StructField("r3",IntegerType(),True), \
        StructField("r4",IntegerType(),True), \
        StructField("r1_ele",FloatType(),True), \
        StructField("r2_ele",FloatType(),True), \
        StructField("r3_ele",FloatType(),True), \
        StructField("r4_ele",FloatType(),True) \
      ])

    df_tetra_origin = spark.read.format("csv") \
          .option("header", False) \
          .schema(schema_tetra_origin)\
          .load(hdfs_tetra_origin)
        
    return df_tetra_origin


# read tetrahedra
hdfs_tetra_origin = directory + "/" + tin_filename + '_filtra_tetra_sort.csv'

df_tetra_order = read_tetra_order(hdfs_tetra_origin)
df_tetra_order.printSchema()

Number of partitions for df_tetra_origin: 800
root
 |-- tetra_order: integer (nullable = true)
 |-- r1: integer (nullable = true)
 |-- r2: integer (nullable = true)
 |-- r3: integer (nullable = true)
 |-- r4: integer (nullable = true)
 |-- r1_ele: float (nullable = true)
 |-- r2_ele: float (nullable = true)
 |-- r3_ele: float (nullable = true)
 |-- r4_ele: float (nullable = true)



# compute TV relation

### Method: local method

In [5]:
# function to get TV relation
def get_TV(df_tetra_order):
    df_TV = df_tetra_order.withColumn("tetra", sort_array(F.array("r1", "r2", "r3", "r4"), False))
    df_TV = df_TV.select('tetra', 'r1', 'r2', 'r3', 'r4')
    
    return df_TV

In [6]:
# df_tetra_order.persist(StorageLevel.MEMORY_ONLY)

df_TV = get_TV(df_tetra_order)

df_TV.printSchema()

root
 |-- tetra: array (nullable = false)
 |    |-- element: integer (containsNull = true)
 |-- r1: integer (nullable = true)
 |-- r2: integer (nullable = true)
 |-- r3: integer (nullable = true)
 |-- r4: integer (nullable = true)



In [7]:
t0 = time.time()

num_row = df_TV.count()

t1 = time.time()
print("Time cost:", t1 - t0)
print("number of rows:", num_row)

Time cost: 45.27763533592224
number of rows: 1214424036


In [8]:
t0 = time.time()

file_TV = directory + '/' + tin_filename + '_TV.parquet'

df_TV.write.parquet(file_TV)

t1 = time.time()
print("time cost:", t1-t0)

time cost: 57.211012840270996


# compute TE relation

### Method: local method

In [5]:
# function to get TE relation
def get_TE_from_T(df_tetra_order):
    df_TE = df_tetra_order.withColumn("tetra", sort_array(F.array("r1", "r2", "r3", "r4"), False)).withColumn("e1", sort_array(F.array("r1", "r2",), False)).withColumn("e2", sort_array(F.array("r1", "r3",), False)).withColumn("e3", sort_array(F.array("r1", "r4",), False)).withColumn("e4", sort_array(F.array("r2", "r3",), False)).withColumn("e5", sort_array(F.array("r2", "r4",), False)).withColumn("e6", sort_array(F.array("r3", "r4",), False))
    df_TE = df_TE.select('tetra', 'e1', 'e2', 'e3', 'e4', 'e5', 'e6')
    
    return df_TE

In [6]:
# df_TE.persist(StorageLevel.MEMORY_ONLY)

df_TE = get_TE_from_T(df_tetra_order)

df_TE.printSchema()

root
 |-- tetra: array (nullable = false)
 |    |-- element: integer (containsNull = true)
 |-- e1: array (nullable = false)
 |    |-- element: integer (containsNull = true)
 |-- e2: array (nullable = false)
 |    |-- element: integer (containsNull = true)
 |-- e3: array (nullable = false)
 |    |-- element: integer (containsNull = true)
 |-- e4: array (nullable = false)
 |    |-- element: integer (containsNull = true)
 |-- e5: array (nullable = false)
 |    |-- element: integer (containsNull = true)
 |-- e6: array (nullable = false)
 |    |-- element: integer (containsNull = true)



In [7]:
t0 = time.time()

num_row = df_TE.count()

t1 = time.time()
print("Time cost:", t1 - t0)
print("number of rows:", num_row)

Time cost: 123.58114004135132
number of rows: 9715202480


In [8]:
t0 = time.time()

file_TE = directory + '/' + tin_filename + '_TE.parquet'

df_TE.write.parquet(file_TE)

t1 = time.time()
print("time cost:", t1-t0)

time cost: 480.0889575481415


# compute TF relation

### Method: local method

In [5]:
# function to get TF relation
def get_TF_from_T(df_tetra_order):
    df_TF = df_tetra_order.withColumn("tetra", sort_array(F.array("r1", "r2", "r3", "r4"), False)).withColumn("f1", sort_array(F.array("r1", "r2", "r3"), False)).withColumn("f2", sort_array(F.array("r1", "r2", "r4"), False)).withColumn("f3", sort_array(F.array("r1", "r3", "r4"), False)).withColumn("f4", sort_array(F.array("r2", "r3", "r4"), False))
    df_TF = df_TF.select('tetra', 'f1', 'f2', 'f3', 'f4')
    
    return df_TF

In [6]:
# df_TF.persist(StorageLevel.MEMORY_ONLY)

df_TF = get_TF_from_T(df_tetra_order)

df_TF.printSchema()

root
 |-- tetra: array (nullable = false)
 |    |-- element: integer (containsNull = true)
 |-- f1: array (nullable = false)
 |    |-- element: integer (containsNull = true)
 |-- f2: array (nullable = false)
 |    |-- element: integer (containsNull = true)
 |-- f3: array (nullable = false)
 |    |-- element: integer (containsNull = true)
 |-- f4: array (nullable = false)
 |    |-- element: integer (containsNull = true)



In [7]:
t0 = time.time()

num_row = df_TF.count()

t1 = time.time()
print("Time cost:", t1 - t0)
print("number of rows:", num_row)

Time cost: 177.00683879852295
number of rows: 9715202480


In [8]:
t0 = time.time()

file_TF = directory + '/' + tin_filename + '_TF.parquet'

df_TF.write.parquet(file_TF)

t1 = time.time()
print("time cost:", t1-t0)

time cost: 409.2017467021942


# compute FE relation

### Method 1: pure global

In [5]:
# function to get TF relation
def get_TF_from_T(df_tetra_order):
    df_TF = df_tetra_order.withColumn("tetra", sort_array(F.array("r1", "r2", "r3", "r4"), False)).withColumn("f1", sort_array(F.array("r1", "r2", "r3"), False)).withColumn("f2", sort_array(F.array("r1", "r2", "r4"), False)).withColumn("f3", sort_array(F.array("r1", "r3", "r4"), False)).withColumn("f4", sort_array(F.array("r2", "r3", "r4"), False))
    df_TF = df_TF.select('tetra', 'f1', 'f2', 'f3', 'f4')
    
    return df_TF

In [6]:
# get faces from DF_T

df_TF = get_TF_from_T(df_tetra_order)

df_F1 = df_TF.select('f1')
df_F2 = df_TF.select('f2')
df_F3 = df_TF.select('f3')
df_F4 = df_TF.select('f4')

df_Faces = df_F1.union(df_F2).union(df_F3).union(df_F4)
df_Faces = df_Faces.withColumnRenamed('f1', 'face')
df_Faces = df_Faces.distinct()
df_Faces.printSchema()

root
 |-- face: array (nullable = false)
 |    |-- element: integer (containsNull = true)



In [7]:
df_FE = df_Faces.withColumn('e1', F.array(df_Faces.face[0], df_Faces.face[1])).withColumn('e2', F.array(df_Faces.face[0], df_Faces.face[2])).withColumn('e3', F.array(df_Faces.face[1], df_Faces.face[2]))
df_FE.printSchema()

root
 |-- face: array (nullable = false)
 |    |-- element: integer (containsNull = true)
 |-- e1: array (nullable = false)
 |    |-- element: integer (containsNull = true)
 |-- e2: array (nullable = false)
 |    |-- element: integer (containsNull = true)
 |-- e3: array (nullable = false)
 |    |-- element: integer (containsNull = true)



In [8]:
t0 = time.time()

num_row = df_FE.count()

t1 = time.time()
print("Time cost:", t1 - t0)
print("number of rows:", num_row)

Time cost: 49.48271083831787
number of rows: 201406060


### Method 2: symmetric global (get FE from EF)

##### load the pre-computed EF relation

In [ ]:
# read a parquet file from hdfs
file_EF = directory + '/' + tin_filename + '_EF.parquet'

df_EF = spark.read.parquet(file_EF)
df_EF.printSchema()

In [ ]:
t0 = time.time()

num_row = df_EF.count()

t1 = time.time()
print("Time cost:", t1 - t0)
print("number of rows:", num_row)

In [ ]:
# duplicates of faces are included in this method; we can use distinct() to remvoe duplicates
df_FE_init = df_EF.select(explode("EF").alias("face"), "Edge").na.drop()
df_FE_init.printSchema()

In [ ]:
df_FE = df_FE_init.groupBy('face').agg(collect_set('Edge').alias('FE'))

df_FE = df_FE.withColumnRenamed('face', 'Face')
df_FE.printSchema()

In [ ]:
t0 = time.time()

num_row = df_FE.count()

t1 = time.time()
print("Time cost:", t1 - t0)
print("number of rows:", num_row)

### Method 3: local method (get FE from VT)

##### load the pre-computed VT relation

In [4]:
# read a parquet file from hdfs
file_VT = directory + '/' + tin_filename + '_VT.parquet'

df_VT = spark.read.parquet(file_VT)
df_VT.printSchema()

root
 |-- Ver: integer (nullable = true)
 |-- VT: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)



In [5]:
t0 = time.time()

num_row = df_VT.count()

t1 = time.time()
print("Time cost:", t1 - t0)
print("number of rows:", num_row)

Time cost: 11.954629898071289
number of rows: 1792989718


In [6]:
# get FV for triangles incident in v_i and having v_i as its largest extreme vertex
def get_FE_from_VT(Ver, VT):
    FE = {}
    if VT:
        # multi_F = set()
        for tetra in VT:
            index_ver = tetra.index(Ver)
            if index_ver == 1: # the second vertex is the current vertex, e.g., [3,2,1,0], value 2 is the expected one                
                tri_0 = [tetra[1], tetra[2], tetra[3]]
                if tuple(tri_0) not in FE:
                    FE[tuple(tri_0)] = [[tetra[1], tetra[2]], [tetra[2], tetra[3]], [tetra[1], tetra[3]]]
                
            if index_ver == 0: # the second vertex is the current vertex, e.g., [3,2,1,0], value 3 is the expected one
                tri_0 = [tetra[0], tetra[1], tetra[2]]
                if tuple(tri_0) not in FE:
                    FE[tuple(tri_0)] = [[tetra[0], tetra[1]], [tetra[0], tetra[2]], [tetra[1], tetra[2]]]
                
                tri_1 = [tetra[0], tetra[1], tetra[3]]
                if tuple(tri_1) not in FE:
                    FE[tuple(tri_1)] = [[tetra[0], tetra[1]], [tetra[0], tetra[3]], [tetra[1], tetra[3]]]
                
                tri_2 = [tetra[0], tetra[2], tetra[3]]
                if tuple(tri_2) not in FE:
                    FE[tuple(tri_2)] = [[tetra[0], tetra[2]], [tetra[0], tetra[3]], [tetra[2], tetra[3]]]
    return FE

get_FE_from_VT_star_udf = udf(get_FE_from_VT, MapType(ArrayType(IntegerType()), ArrayType(ArrayType(IntegerType()))))

In [7]:
# store the FT relation within VT star
df_FE = df_VT.withColumn("FV", get_FE_from_VT_star_udf(df_VT.Ver, df_VT.VT))
df_FE = df_FE.select("Ver", "FV")
df_FE.printSchema()

root
 |-- Ver: integer (nullable = true)
 |-- FV: map (nullable = true)
 |    |-- key: array
 |    |    |-- element: integer (containsNull = true)
 |    |-- value: array (valueContainsNull = true)
 |    |    |-- element: array (containsNull = true)
 |    |    |    |-- element: integer (containsNull = true)



In [ ]:
t0 = time.time()

num_row = df_FE.count()

t1 = time.time()
print("Time cost:", t1 - t0)
print("number of rows:", num_row)

# compute FV relation

### Method 1: pure global

In [5]:
# function to get TF relation
def get_TF_from_T(df_tetra_order):
    df_TF = df_tetra_order.withColumn("tetra", sort_array(F.array("r1", "r2", "r3", "r4"), False)).withColumn("f1", sort_array(F.array("r1", "r2", "r3"), False)).withColumn("f2", sort_array(F.array("r1", "r2", "r4"), False)).withColumn("f3", sort_array(F.array("r1", "r3", "r4"), False)).withColumn("f4", sort_array(F.array("r2", "r3", "r4"), False))
    df_TF = df_TF.select('tetra', 'f1', 'f2', 'f3', 'f4')
    
    return df_TF

In [6]:
# get faces from DF_T

df_TF = get_TF_from_T(df_tetra_order)

df_F1 = df_TF.select('f1')
df_F2 = df_TF.select('f2')
df_F3 = df_TF.select('f3')
df_F4 = df_TF.select('f4')

df_Faces = df_F1.union(df_F2).union(df_F3).union(df_F4)
df_Faces = df_Faces.withColumnRenamed('f1', 'face')
df_Faces = df_Faces.distinct()
df_Faces.printSchema()

root
 |-- face: array (nullable = false)
 |    |-- element: integer (containsNull = true)



In [7]:
df_FV = df_Faces.withColumn('r1', df_Faces.face[0]).withColumn('r2', df_Faces.face[1]).withColumn('r3', df_Faces.face[2])
df_FV.printSchema()

root
 |-- face: array (nullable = false)
 |    |-- element: integer (containsNull = true)
 |-- r1: integer (nullable = true)
 |-- r2: integer (nullable = true)
 |-- r3: integer (nullable = true)



In [8]:
t0 = time.time()

num_row = df_FV.count()

t1 = time.time()
print("Time cost:", t1 - t0)
print("number of rows:", num_row)

Time cost: 836.0797238349915
number of rows: 19434269798


### Method 2: symmetric global (get FV from VF)

##### load the pre-computed VF relation

In [4]:
# read a parquet file from hdfs
file_VF = directory + '/' + tin_filename + '_VF.parquet'

df_VF = spark.read.parquet(file_VF)
df_VF.printSchema()

root
 |-- Ver: integer (nullable = true)
 |-- VF: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)



In [5]:
t0 = time.time()

num_row = df_VF.count()

t1 = time.time()
print("Time cost:", t1 - t0)
print("number of rows:", num_row)

Time cost: 18.70658826828003
number of rows: 1792989718


In [6]:
# duplicates of faces are included in this method; we can use distinct() to remvoe duplicates
df_Faces = df_VF.select(explode("VF").alias("face")).na.drop()
df_Faces = df_Faces.distinct()
df_Faces.printSchema()

root
 |-- face: array (nullable = true)
 |    |-- element: integer (containsNull = true)



In [7]:
df_FV = df_Faces.withColumn('r1', df_Faces.face[0]).withColumn('r2', df_Faces.face[1]).withColumn('r3', df_Faces.face[2])
df_FV.printSchema()

root
 |-- face: array (nullable = true)
 |    |-- element: integer (containsNull = true)
 |-- r1: integer (nullable = true)
 |-- r2: integer (nullable = true)
 |-- r3: integer (nullable = true)



In [8]:
t0 = time.time()

num_row = df_FV.count()

t1 = time.time()
print("Time cost:", t1 - t0)
print("number of rows:", num_row)

Time cost: 980.7785394191742
number of rows: 19434269798


### Method 3: local method (get FE from VT)

In [4]:
# read a parquet file from hdfs
file_VT = directory + '/' + tin_filename + '_VT.parquet'

df_VT = spark.read.parquet(file_VT)
df_VT.printSchema()

root
 |-- Ver: integer (nullable = true)
 |-- VT: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)



In [5]:
t0 = time.time()

num_row = df_VT.count()

t1 = time.time()
print("Time cost:", t1 - t0)
print("number of rows:", num_row)

Time cost: 11.17226767539978
number of rows: 17372993


In [6]:
# get FV for triangles incident in v_i and having v_i as its largest extreme vertex
def get_FV_from_VT(Ver, VT):
    FV = {}
    if VT:
        # multi_F = set()
        for tetra in VT:
            index_ver = tretra.index(Ver)
            if index_ver == 1: # the second vertex is the current vertex, e.g., [3,2,1,0], value 2 is the expected one, we want triangle [2,1,0]
                tri_0 = [tetra[1], tetra[2], tetra[3]]
                if tuple(tri_0) not in FV:
                    FV[tuple(tri_0)] = tri_0
                
            if index_ver == 0: # the second vertex is the current vertex, e.g., [3,2,1,0], value 2 is the expected one, we want triangle [2,1,0]
                tri_0 = [tetra[0], tetra[1], tetra[2]]
                if tuple(tri_0) not in FV:
                    FV[tuple(tri_0)] = tri_0
                
                tri_1 = [tetra[0], tetra[1], tetra[3]]
                if tuple(tri_1) not in FV:
                    FV[tuple(tri_1)] = tri_1
                
                tri_2 = [tetra[0], tetra[2], tetra[3]]
                if tuple(tri_2) not in FV:
                    FV[tuple(tri_2)] = tri_2
    return FV

get_FV_from_VT_star_udf = udf(get_FV_from_VT, MapType(ArrayType(IntegerType()), ArrayType(IntegerType())))

In [7]:
# store the FT relation within VT star
df_FV = df_VT.withColumn("FV", get_FV_from_VT_star_udf(df_VT.Ver, df_VT.VT))
df_FV = df_FV.select("Ver", "FV")
df_FV.printSchema()

root
 |-- Ver: integer (nullable = true)
 |-- FV: map (nullable = true)
 |    |-- key: array
 |    |    |-- element: integer (containsNull = true)
 |    |-- value: array (valueContainsNull = true)
 |    |    |-- element: integer (containsNull = true)



In [8]:
t0 = time.time()

num_row = df_FV.count()

t1 = time.time()
print("Time cost:", t1 - t0)
print("number of rows:", num_row)

Time cost: 4.29792594909668
number of rows: 17372993


# compute EV relation

### Method 1: pure global

In [5]:
# function to get TE relation
def get_TE_from_T(df_tetra_order):
    df_TE = df_tetra_order.withColumn("tetra", sort_array(F.array("r1", "r2", "r3", "r4"), False)).withColumn("e1", sort_array(F.array("r1", "r2",), False)).withColumn("e2", sort_array(F.array("r1", "r3",), False)).withColumn("e3", sort_array(F.array("r1", "r4",), False)).withColumn("e4", sort_array(F.array("r2", "r3",), False)).withColumn("e5", sort_array(F.array("r2", "r4",), False)).withColumn("e6", sort_array(F.array("r3", "r4",), False))
    df_TE = df_TE.select('tetra', 'e1', 'e2', 'e3', 'e4', 'e5', 'e6')
    
    return df_TE

In [6]:
# get edges from DF_T

df_TE = get_TE_from_T(df_tetra_order)

df_E1 = df_TE.select('e1')
df_E2 = df_TE.select('e2')
df_E3 = df_TE.select('e3')
df_E4 = df_TE.select('e4')
df_E5 = df_TE.select('e5')
df_E6 = df_TE.select('e6')

df_Edges = df_E1.union(df_E2).union(df_E3).union(df_E4).union(df_E5).union(df_E6)
df_Edges = df_Edges.withColumnRenamed('e1', 'edge')
df_Edges = df_Edges.distinct()
df_Edges.printSchema()

root
 |-- edge: array (nullable = false)
 |    |-- element: integer (containsNull = true)



In [7]:
df_EV = df_Edges.withColumn('r1', df_Edges.edge[0]).withColumn('r2', df_Edges.edge[1])
df_EV.printSchema()

root
 |-- edge: array (nullable = false)
 |    |-- element: integer (containsNull = true)
 |-- r1: integer (nullable = true)
 |-- r2: integer (nullable = true)



In [8]:
t0 = time.time()

num_row = df_EV.count()

t1 = time.time()
print("Time cost:", t1 - t0)
print("number of rows:", num_row)

Time cost: 1018.6769051551819
number of rows: 11511582639


### Method 2: symmetric global (get EV from VE)

##### load the pre-computed VE relation

In [4]:
# read a parquet file from hdfs
file_VE = directory + '/' + tin_filename + '_VE.parquet'

df_VE = spark.read.parquet(file_VE)
df_VE.printSchema()

root
 |-- Ver: integer (nullable = true)
 |-- VE: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)



In [5]:
t0 = time.time()

num_row = df_VE.count()

t1 = time.time()
print("Time cost:", t1 - t0)
print("number of rows:", num_row)

Time cost: 19.424075841903687
number of rows: 1792989718


In [6]:
# duplicates of faces are included in this method; we can use distinct() to remvoe duplicates
df_Edges = df_VE.select(explode("VE").alias("edge"))
df_Edges = df_Edges.distinct()
df_Edges.printSchema()

root
 |-- edge: array (nullable = true)
 |    |-- element: integer (containsNull = true)



In [7]:
df_EV = df_Edges.withColumn('r1', df_Edges.edge[0]).withColumn('r2', df_Edges.edge[1])
df_EV.printSchema()

root
 |-- edge: array (nullable = true)
 |    |-- element: integer (containsNull = true)
 |-- r1: integer (nullable = true)
 |-- r2: integer (nullable = true)



In [8]:
t0 = time.time()

df_EV.count()

t1 = time.time()
print("Time cost:", t1 - t0)

Time cost: 432.9955496788025


### Method 3: local method (get EV from VT)

In [4]:
# read a parquet file from hdfs
file_VT = directory + '/' + tin_filename + '_VT.parquet'

df_VT = spark.read.parquet(file_VT)
df_VT.printSchema()

root
 |-- Ver: integer (nullable = true)
 |-- VT: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: integer (containsNull = true)



In [5]:
t0 = time.time()

num_row = df_VT.count()

t1 = time.time()
print("Time cost:", t1 - t0)
print("number of rows:", num_row)

Time cost: 14.957468509674072
number of rows: 1792989718


In [6]:
# get EV for edges incident in v_i and having v_i as its largest extreme vertex
def get_EV_from_VT(Ver, VT):
    EV = {}
    if VT:
        for tetra in VT:
            index_ver = tretra.index(Ver)
            if index_ver == 2: # the second vertex is the current vertex, e.g., [3,2,1,0], value 1 is the expected one
                e_0 = [tetra[2], tetra[3]]
                if tuple(e_0) not in EV:
                    EV[tuple(e_0)] = e_0
                
            if index_ver == 1: # the second vertex is the current vertex, e.g., [3,2,1,0], value 2 is the expected one
                e_0 = [tetra[1], tetra[2]]
                if tuple(e_0) not in EV:
                    EV[tuple(e_0)] = e_0
                
                e_1 = [tetra[1], tetra[3]]
                if tuple(e_1) not in EV:
                    EV[tuple(e_1)] = e_1
                
            if index_ver == 0: # the second vertex is the current vertex, e.g., [3,2,1,0], value 3 is the expected one
                e_0 = [tetra[0], tetra[1]]
                if tuple(e_0) not in EV:
                    EV[tuple(e_0)] = e_0
                
                e_1 = [tetra[0], tetra[2]]
                if tuple(e_1) not in EV:
                    EV[tuple(e_1)] = e_1
                
                e_2 = [tetra[0], tetra[3]]
                if tuple(e_2) not in EV:
                    EV[tuple(e_2)] = e_2
    return EV

get_EV_from_VT_star_udf = udf(get_EV_from_VT, MapType(ArrayType(IntegerType()), ArrayType(IntegerType())))

In [7]:
# store the FT relation within VT star
df_EV = df_VT.withColumn("EV", get_EV_from_VT_star_udf(df_VT.Ver, df_VT.VT))
df_EV = df_EV.select("Ver", "EV")
df_EV.printSchema()

root
 |-- Ver: integer (nullable = true)
 |-- EV: map (nullable = true)
 |    |-- key: array
 |    |    |-- element: integer (containsNull = true)
 |    |-- value: array (valueContainsNull = true)
 |    |    |-- element: integer (containsNull = true)



In [8]:
t0 = time.time()

num_row = df_EV.count()

t1 = time.time()
print("Time cost:", t1 - t0)
print("number of rows:", num_row)

Time cost: 7.530542612075806
number of rows: 1792989718
